# Cosinor analysis with pyActigraphy
This notebook illustrates how to perform a single-component Cosinor analysis on actigraphy data.

The idea of a Cosinor analysis is to estimate some key parameters of the actigraphy count series by fitting these data with a (co)sine curve with a period of 24h:
$$ Y(t) = M + A*cos(\frac{2\pi}{T}*t + \phi)$$
where M is the MESOR (Midline Statistic Of Rhythm), A is the amplitude of the oscillations, T is the period and $\phi$ is the acrophase.

Then, the fit procedure provides estimates of these parameters which can then help to characterize the 24h rest-activity rhythm of an individual.

However, despite its popularity in circadian rhythm analyses, the Cosinor analysis exhibits some short-comings when dealing with non-stationary signals such as actigraphy signals. Intuitively, such model with a fixed period, amplitude and offset cannot accomodate signals with:

* rest-activity cycles that do not follow a perfect 24h period;
* an overall daily activity that changes on a day-by-day basis;
* an overall activity trend that evolves with time.

For more informations on the topic:

Leise, T. L., & Harrington, M. E. (2011). *Wavelet-Based Time Series Analysis of Circadian Rhythms*. Journal of Biological Rhythms, 26(5), 454–463. https://doi.org/10.1177/0748730411416330

For a nice comparison of several techniques, including Cosinor and SSA (available in pyActigraphy):

Fossion, R., Rivera, A. L., Toledo-Roy, J. C., Ellis, J., & Angelova, M. (2017). *Multiscale adaptive analysis of circadian rhythms and intradaily variability: Application to actigraphy time series in acute insomnia subjects*. PLOS ONE, 12(7), e0181762. https://doi.org/10.1371/journal.pone.0181762

## Import packages and input data

First, load your favourite python packages:

In [ ]:
import pyActigraphy

In [ ]:
import plotly.graph_objects as go

Let us now define the path to the example data files contained in the pyActigraphy package:

In [ ]:
import os
fpath = os.path.join(os.path.dirname(pyActigraphy.__file__),'tests/data/')

## Cosinor analysis

Now, simply import the Cosinor module:

In [ ]:
from pyActigraphy.analysis import Cosinor

... and define a Cosinor object:

In [ ]:
cosinor = Cosinor()

### Initial fit values

By default, the initial values of the cosine fit functions are the following:

In [ ]:
cosinor.fit_initial_params.pretty_print()

These parameters will be estimated from the data. However, in order to ease the fit procedure, it makes sense to start from reasonable values.
For example, the initial value for the period is set to 1440, the number of minutes per day. It makes only sense for data with a 1-min sampling frequency. 

However, it is trivial to change these intial values. 

For example, let's change the initial value of the period to 2880, in case you would like to analyse data with a 30-sec sampling frequency:

In [ ]:
cosinor.fit_initial_params['Period'].value = 2880

In [ ]:
cosinor.fit_initial_params.pretty_print()

### Free and fixed fit parameters

By default, none of the fit parameters are fixed and will be estimated from the data. However, in some cases, it might be convenient to fix the initial of a parameter and not let it vary during the fit procedure.

It might be the case of the period for instance. Let us fix it to 1440.

In [ ]:
cosinor.fit_initial_params['Period'].value = 1440

In [ ]:
cosinor.fit_initial_params['Period'].vary = False

In [ ]:
cosinor.fit_initial_params.pretty_print()

It is also possible to apply lower and upper bounds to these parameters.

For more informations about these parameters, please see: https://lmfit.github.io/lmfit-py/parameters.html

### Analysis of a single subject

Read data from an example file...

In [ ]:
raw = pyActigraphy.io.read_raw_awd(fpath+'example_01.AWD', start_time='1918-01-24 08:30:00', period="7D")

... and plot it for visual inspection:

In [ ]:
go.Figure(go.Scatter(x=raw.data.index.astype(str),y=raw.data))

To perform a cosinor analysis with the pyActigraphy package, it is as simple as:

In [ ]:
results = cosinor.fit(raw.data, verbose=True) # Set verbose to True to print the fit output

#### Fit results

To access the best fit parameter values:

In [ ]:
results.params['Mesor'].value

It is also possible to transform them to a dictionary:

In [ ]:
results.params.valuesdict()

The `results` object contains also informations about the goodness-of-fit:

In [ ]:
results.aic # Akaike information criterium

In [ ]:
results.redchi # Reduced Chi^2 

More informations about the list of the informations accessible via the `results` object: https://lmfit.github.io/lmfit-py/fitting.html#minimizerresult-the-optimization-result

#### Fit result visualization

If you are interested in visualizing the cosinor fit that best approximates the data, the `Cosinor` class implements a convinient function:

In [ ]:
help(cosinor.best_fit)

In [ ]:
best_fit = cosinor.best_fit(raw.data, results.params)

In [ ]:
go.Figure(
    data=[
        go.Scatter(x=raw.data.index.astype(str),y=raw.data,name='Raw data'),
        go.Scatter(x=best_fit.index.astype(str),y=best_fit,name='Best fit')
    ]
)

### Batch analysis

While it is sometimes necessary to perform a cosinor analysis of a single subject to get used with the procedure and get "a feeling" about the data, it is often not pratical to follow the same procedure to analyse multiple subjects.

Fortunately, `pyActigraphy` implements several convenient functions to ease a batch analysis.

Let us first read a batch of files, using the example files included the `pyActigraphy` package for demonstration purposes:

In [ ]:
readers = pyActigraphy.io.read_raw(fpath+'example_*.AWD', reader_type='AWD')

In [ ]:
len(readers.readers)

6 files have been found and read by the `readers` object.

Read and apply a SST log (See https://ghammad.github.io/pyActigraphy/pyActigraphy-SSt-log.html for more informations):

In [ ]:
readers.read_sst_log(fpath+'example_sstlog.csv')

In [ ]:
readers.apply_sst(verbose=True)

Now, let perform a cosinor analysis on each of these files, in parallel, using 3 cpu's in order to speed up the computations.

In [ ]:
results_batch = cosinor.fit_reader(readers, n_jobs=3, prefer='threads') # prefer='threads': add that parameter if running on Mac OS. 

For convenience, the output results are formatted as a Pandas.DataFrame (https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe) in order to ease further analysis.

In [ ]:
results_batch

To write these results to an output file, it is as simple as: ```results_batch.to_csv('myfile.csv')```.

More info on `to_csv`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html?highlight=to_csv#pandas-dataframe-to-csv

Et voilà! Easy, isn't it?